# SOLAS Interactive Notebook

This notebook provides an interactive interface for running the SOLAS pipeline with custom parameters.

---

**Usage:**
Run each cell **one by one** (using the Play button) and follow the instructions presented. Some cells contain interactive widgets (Configuration, Audio Upload) that require your input before proceeding to the next step.

---

**⚠️ IMPORTANT: GPU Runtime Required**

This notebook requires a **GPU runtime** for optimal performance
- The notebook should default to GPU, but verify in the top-right corner that it shows "GPU" or "T4 GPU"
- If necessary, go to **Runtime → Change runtime type** and select **GPU** (T4 is sufficient)

In [ ]:
# @title ### Setup Environment
# @markdown Check environment and install dependencies if needed.

import sys
import subprocess
import os
from pathlib import Path
from IPython.display import display, HTML

# ============================================================================
# Download the SOLAS repository
# ============================================================================
if Path('SOLAS').exists():
    # Pull latest changes
    subprocess.run(['git', 'pull'], check=True, cwd='SOLAS')
else:
    subprocess.run(['git', 'clone', 'https://github.com/andrecarini/SOLAS.git'], check=True)

sys.path.insert(0, 'SOLAS')

# ============================================================================
# Run basic checks
# ============================================================================
if 'RESTART_NEEDED' in globals() and RESTART_NEEDED:
    from library import show_restart_warning
    show_restart_warning()

from library import check_colab_environment
check_colab_environment()

# ============================================================================
# Setup the environment
# ============================================================================
from library import setup_environment_with_progress
result = setup_environment_with_progress()

# The setup_complete.html template already includes the restart warning if needed
# So we just set the flags without calling show_restart_warning() again
if result.get('restart_needed'):
    RESTART_NEEDED = True
else:
    RESTART_NEEDED = False
    SETUP_COMPLETE = True

In [ ]:
# @title ### Configuration
# @markdown Configure pipeline parameters using the widgets below.

# ============================================================================
# Run basic checks
# ============================================================================
if 'RESTART_NEEDED' in globals() and RESTART_NEEDED:
    from library import show_restart_warning
    show_restart_warning()

if 'SETUP_COMPLETE' not in globals() or not SETUP_COMPLETE:
    raise RuntimeError("Setup not completed. Please run the cell above to complete environment setup before configuring the pipeline.")

from library import check_colab_environment
check_colab_environment()

# ============================================================================
# Display configuration interface
# ============================================================================
from library import create_config_widgets, load_template
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Get widget settings (for storing values)
SOLAS_SETTINGS = create_config_widgets()

# Show configuration ready message (nicely styled infobox)
display(HTML(load_template('config_ready.html')))

# Display the pre-built config box
display(SOLAS_SETTINGS["config_box"])

# ============================================================================
# Confirmation Button
# ============================================================================
def on_confirm_click(b):
    """Handle configuration confirmation button click."""
    confirm_button.disabled = True
    confirm_button.description = "Configuration Confirmed"
    confirm_button.button_style = "success"

confirm_button = widgets.Button(
    description="Confirm Configuration & Continue",
    button_style="primary",
    layout=widgets.Layout(width="300px", height="40px")
)
confirm_button.on_click(on_confirm_click)
display(confirm_button)

In [ ]:
# @title ### Upload Input Audio
# @markdown Upload the lecture audio file to transcribe.

import sys
from pathlib import Path

# ============================================================================
# Run basic checks
# ============================================================================
if 'RESTART_NEEDED' in globals() and RESTART_NEEDED:
    from library import show_restart_warning
    show_restart_warning()

if 'SETUP_COMPLETE' not in globals() or not SETUP_COMPLETE:
    raise RuntimeError("Setup not completed. Please run the first cell to complete environment setup.")

if 'SOLAS_SETTINGS' not in globals():
    from library import show_config_warning
    show_config_warning()

from library import check_colab_environment
COLAB = check_colab_environment()

# ============================================================================
# Audio Upload Interface
# ============================================================================
if COLAB:
    from google.colab import files as colab_files
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    from library import load_template
    
    # Show info message (nicely styled infobox)
    display(HTML(load_template('audio_selection_ready.html')))
    
    # Setup paths
    BASE_DIR = Path('/content') if Path('/content').exists() else Path.cwd()
    SOLAS_DIR = BASE_DIR / 'SOLAS'
    input_audio_text = SOLAS_SETTINGS["input_audio_text"]
    
    # Sample audio file mapping - Updated to .ogg for Portuguese samples
    SAMPLE_AUDIO_MAP = {
        'Use sample audio (short)': 'short.ogg',
        'Use sample audio (medium)': 'medium.ogg',
        'Use sample audio (long)': 'long.ogg'
    }
    
    # Create radio button for selection
    audio_choice = widgets.RadioButtons(
        options=list(SAMPLE_AUDIO_MAP.keys()) + ['Upload my own audio'],
        value='Use sample audio (short)',
        description='Audio source:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='500px')
    )
    
    # Create output area for upload/status messages
    output_area = widgets.Output()
    
    # Confirm button (matching Configuration style)
    confirm_btn = widgets.Button(
        description='Confirm Audio Selection',
        button_style='primary',
        layout=widgets.Layout(width='300px', height='40px')
    )
    
    def on_confirm(btn):
        """Handle audio selection confirmation."""
        choice = audio_choice.value
        
        with output_area:
            clear_output()
            
            if choice in SAMPLE_AUDIO_MAP:
                sample_path = SOLAS_DIR / 'input_audio_samples' / SAMPLE_AUDIO_MAP[choice]
                if sample_path.exists():
                    input_audio_text.value = str(sample_path)
                    # Success - disable button
                    confirm_btn.disabled = True
                    confirm_btn.description = "Audio Selected"
                    confirm_btn.button_style = "success"
                else:
                    display(HTML(f'<span style="color: var(--color-error);">Sample file not found: {sample_path}</span>'))
            else:
                display(HTML('<span style="color: var(--color-text-secondary);">Select an audio file to upload...</span>'))
                try:
                    uploaded = colab_files.upload()
                    if uploaded:
                        name = next(iter(uploaded.keys()))
                        data = uploaded[name]
                        save_path = BASE_DIR / name
                        with open(save_path, 'wb') as f:
                            f.write(data)
                        input_audio_text.value = str(save_path)
                        clear_output()
                        display(HTML(f'<span style="color: var(--color-success);">Uploaded: {save_path}</span>'))
                        # Success - disable button
                        confirm_btn.disabled = True
                        confirm_btn.description = "Audio Selected"
                        confirm_btn.button_style = "success"
                    else:
                        clear_output()
                        display(HTML('<span style="color: var(--color-error);">No file was uploaded.</span>'))
                except Exception as e:
                    clear_output()
                    display(HTML(f'<span style="color: var(--color-error);">Error: {e}</span>'))
    
    confirm_btn.on_click(on_confirm)
    
    # Display widgets
    display(widgets.VBox([audio_choice, confirm_btn, output_area]))
    
else:
    # Fallback for non-Colab environments
    from library import create_audio_upload_widget
    from IPython.display import display
    upload_widget = create_audio_upload_widget(SOLAS_SETTINGS)
    display(upload_widget)

In [ ]:
# @title ### Upload Host Voices
# @markdown Select or upload voice samples for the podcast hosts.

import sys
from pathlib import Path

# ============================================================================
# Run basic checks
# ============================================================================
if 'RESTART_NEEDED' in globals() and RESTART_NEEDED:
    from library import show_restart_warning
    show_restart_warning()

if 'SETUP_COMPLETE' not in globals() or not SETUP_COMPLETE:
    raise RuntimeError("Setup not completed. Please run the first cell to complete environment setup.")

if 'SOLAS_SETTINGS' not in globals():
    from library import show_config_warning
    show_config_warning()

from library import check_colab_environment
COLAB = check_colab_environment()

# ============================================================================
# Host Voice Upload Interface
# ============================================================================
if COLAB:
    from google.colab import files as colab_files
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    from library import load_template
    
    # Show info message (nicely styled infobox)
    display(HTML(load_template('host_voice_selection_ready.html')))
    
    # Setup paths
    BASE_DIR = Path('/content') if Path('/content').exists() else Path.cwd()
    SOLAS_DIR = BASE_DIR / 'SOLAS'
    host_a_text = SOLAS_SETTINGS["host_a_text"]
    host_b_text = SOLAS_SETTINGS["host_b_text"]
    
    # Create selection widgets for Host A
    host_a_choice = widgets.RadioButtons(
        options=['Use sample voice (male)', 'Upload my own voice'],
        value='Use sample voice (male)',
        description='Host A:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Create selection widgets for Host B
    host_b_choice = widgets.RadioButtons(
        options=['Use sample voice (female)', 'Upload my own voice'],
        value='Use sample voice (female)',
        description='Host B:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Create output areas
    output_area_a = widgets.Output()
    output_area_b = widgets.Output()
    
    # Confirm button (matching Configuration style)
    confirm_btn = widgets.Button(
        description='Confirm Voice Selections',
        button_style='primary',
        layout=widgets.Layout(width='300px', height='40px')
    )
    
    def on_confirm(btn):
        """Handle host voice selection confirmation."""
        success = True
        
        # Handle Host A
        with output_area_a:
            clear_output()
            choice_a = host_a_choice.value
            
            if choice_a == 'Use sample voice (male)':
                sample_path = SOLAS_DIR / 'TTS_voice_samples' / 'male.wav'
                if sample_path.exists():
                    host_a_text.value = str(sample_path)
                else:
                    display(HTML(f'<span style="color: var(--color-error);">Sample not found: {sample_path}</span>'))
                    success = False
            else:
                display(HTML('<span style="color: var(--color-text-secondary);">Select Host A voice file...</span>'))
                uploaded_a = colab_files.upload()
                if uploaded_a:
                    name_a = next(iter(uploaded_a.keys()))
                    data_a = uploaded_a[name_a]
                    save_path_a = BASE_DIR / name_a
                    with open(save_path_a, 'wb') as f:
                        f.write(data_a)
                    host_a_text.value = str(save_path_a)
                    clear_output()
                    display(HTML(f'<span style="color: var(--color-success);">Uploaded: {save_path_a.name}</span>'))
                else:
                    clear_output()
                    display(HTML('<span style="color: var(--color-error);">No file uploaded for Host A</span>'))
                    success = False
        
        if not success:
            return
        
        # Handle Host B
        with output_area_b:
            clear_output()
            choice_b = host_b_choice.value
            
            if choice_b == 'Use sample voice (female)':
                sample_path = SOLAS_DIR / 'TTS_voice_samples' / 'female.wav'
                if sample_path.exists():
                    host_b_text.value = str(sample_path)
                else:
                    display(HTML(f'<span style="color: var(--color-error);">Sample not found: {sample_path}</span>'))
                    success = False
            else:
                display(HTML('<span style="color: var(--color-text-secondary);">Select Host B voice file...</span>'))
                uploaded_b = colab_files.upload()
                if uploaded_b:
                    name_b = next(iter(uploaded_b.keys()))
                    data_b = uploaded_b[name_b]
                    save_path_b = BASE_DIR / name_b
                    with open(save_path_b, 'wb') as f:
                        f.write(data_b)
                    host_b_text.value = str(save_path_b)
                    clear_output()
                    display(HTML(f'<span style="color: var(--color-success);">Uploaded: {save_path_b.name}</span>'))
                else:
                    clear_output()
                    display(HTML('<span style="color: var(--color-error);">No file uploaded for Host B</span>'))
                    success = False
        
        # If all successful, disable button
        if success:
            confirm_btn.disabled = True
            confirm_btn.description = "Voices Selected"
            confirm_btn.button_style = "success"
    
    confirm_btn.on_click(on_confirm)
    
    # Display widgets with section headers
    display(HTML('<h4 style="margin: 15px 0 10px 0;">Host A Voice</h4>'))
    display(widgets.VBox([host_a_choice, output_area_a]))
    
    display(HTML('<h4 style="margin: 15px 0 10px 0;">Host B Voice</h4>'))
    display(widgets.VBox([host_b_choice, output_area_b]))
    
    display(confirm_btn)
    
else:
    # Fallback for non-Colab environments
    from library import create_host_voice_upload_widget
    from IPython.display import display
    
    host_a_widget = create_host_voice_upload_widget(SOLAS_SETTINGS, "host_a_text")
    host_b_widget = create_host_voice_upload_widget(SOLAS_SETTINGS, "host_b_text")
    display(host_a_widget)
    display(host_b_widget)

In [ ]:
# @title ### Run Pipeline
# @markdown Execute the full SOLAS pipeline with your configured parameters.

import sys
import json
import time
from IPython.display import display, HTML, clear_output, update_display

# ============================================================================
# Run basic checks
# ============================================================================
if 'RESTART_NEEDED' in globals() and RESTART_NEEDED:
    from library import show_restart_warning
    show_restart_warning()

if 'SETUP_COMPLETE' not in globals() or not SETUP_COMPLETE:
    raise RuntimeError("Setup not completed. Please run the first cell to complete environment setup.")

if 'SOLAS_SETTINGS' not in globals():
    from library import show_config_warning
    show_config_warning()

from library import run_pipeline, build_config_from_widgets, display_results, load_template, log_setup, get_verbosity

# ============================================================================
# Build Configuration
# ============================================================================
verbose = get_verbosity()
config = build_config_from_widgets(SOLAS_SETTINGS)

# Only show config in verbose mode
if verbose:
    log_setup("Pipeline configuration:", 'info', verbose)
    config_json = json.dumps(config, indent=2)
    display(HTML(f'<pre style="font-size: 11px; background: var(--color-bg-secondary); padding: 8px; border-radius: 4px;">{config_json}</pre>'))

# ============================================================================
# Create Progress Display (pure HTML, no widgets to avoid CDN notice)
# ============================================================================
display(HTML(load_template('pipeline_start.html')))

# Stage definitions
STAGES = [
    (1, "Loading and preprocessing audio"),
    (2, "Transcribing audio (ASR)"),
    (3, "Translating transcript"),
    (4, "Generating key points summary"),
    (5, "Creating podcast script"),
    (6, "Synthesizing podcast audio (TTS)")
]

# Create initial progress HTML
def create_progress_html(stage_statuses, stage_progress, stage_info_data):
    """Generate complete progress HTML."""
    rows_html = ""
    for stage_num, stage_name in STAGES:
        status = stage_statuses.get(stage_num, 'pending')
        progress_pct = stage_progress.get(stage_num, 0)
        info_text = stage_info_data.get(stage_num, '')
        
        # Status icon and color
        if status == 'pending':
            icon, color = '*', 'var(--color-text-secondary)'
            weight = 'normal'
        elif status == 'active':
            icon, color = '...', 'var(--color-primary)'
            weight = 'bold'
        elif status == 'complete':
            icon, color = 'v', 'var(--color-success)'
            weight = 'normal'
        else:
            icon, color = 'x', 'var(--color-error)'
            weight = 'normal'
        
        # Progress bar color
        if progress_pct == 100:
            bar_color = 'var(--color-success)'
        elif progress_pct > 0:
            bar_color = 'var(--color-primary)'
        else:
            bar_color = 'var(--color-bg-secondary)'
        
        rows_html += f'''
        <div style="display: flex; align-items: center; margin: 4px 0;">
            <div style="width: 380px; color: {color}; font-weight: {weight};">
                {icon} Stage {stage_num}/6: {stage_name}
            </div>
            <div style="width: 180px; margin: 0 0 0 10px;">
                <div style="width: 100%; height: 18px; background: var(--color-bg-secondary); border-radius: 4px; overflow: hidden;">
                    <div style="width: {progress_pct}%; height: 100%; background: {bar_color}; transition: width 0.3s;"></div>
                </div>
            </div>
            <div style="width: 200px; margin: 0 0 0 10px; color: var(--color-text-secondary); font-size: 12px;">
                {info_text}
            </div>
        </div>
        '''
    
    return f'''
    <div style="padding: 10px;">
        <div id="pipeline-header">
            <h3 style="color: var(--color-primary); margin-bottom: 10px;">Running SOLAS Pipeline</h3>
        </div>
        <div id="pipeline-stages">
            {rows_html}
        </div>
    </div>
    '''

# Initialize stage tracking
stage_statuses = {num: 'pending' for num, _ in STAGES}
stage_progress = {num: 0 for num, _ in STAGES}
stage_info_data = {num: '' for num, _ in STAGES}

# Display initial progress
progress_display_id = 'pipeline-progress'
display(HTML(create_progress_html(stage_statuses, stage_progress, stage_info_data)), display_id=progress_display_id)

# ============================================================================
# Progress Callback Function
# ============================================================================
def progress_callback(stage_num, stage_name, progress_pct):
    """Update progress display when called by pipeline."""
    global stage_statuses, stage_progress
    
    # Update status
    if progress_pct < 100:
        stage_statuses[stage_num] = 'active'
    else:
        stage_statuses[stage_num] = 'complete'
    
    # Update progress
    stage_progress[stage_num] = progress_pct
    
    # Update display
    update_display(HTML(create_progress_html(stage_statuses, stage_progress, stage_info_data)), display_id=progress_display_id)

def update_stage_info(stage_num, time_seconds, vram_gb=None):
    """Update stage info with time and VRAM."""
    global stage_info_data
    
    info_parts = [f'{time_seconds:.1f}s']
    if vram_gb is not None and vram_gb > 0:
        info_parts.append(f'{vram_gb:.1f}GB VRAM')
    stage_info_data[stage_num] = ' | '.join(info_parts)
    
    # Update display
    update_display(HTML(create_progress_html(stage_statuses, stage_progress, stage_info_data)), display_id=progress_display_id)

def update_header(message, color):
    """Update the header message."""
    current_html = create_progress_html(stage_statuses, stage_progress, stage_info_data)
    new_html = current_html.replace(
        '<h3 style="color: var(--color-primary); margin-bottom: 10px;">Running SOLAS Pipeline</h3>',
        f'<h3 style="color: {color}; margin-bottom: 10px;">{message}</h3>'
    )
    update_display(HTML(new_html), display_id=progress_display_id)

# ============================================================================
# Run Pipeline with Progress Updates
# ============================================================================
try:
    # Custom wrapper to capture timing/VRAM per stage
    import torch
    
    def get_vram_gb():
        if torch.cuda.is_available():
            return torch.cuda.memory_allocated() / (1024**3)
        return 0
    
    current_stage = [0]  # Use list for closure
    stage_start_time = [time.time()]
    
    def enhanced_progress_callback(stage_num, stage_name, progress_pct):
        # When starting a new stage, record info for previous stage
        if stage_num != current_stage[0] and current_stage[0] > 0:
            elapsed = time.time() - stage_start_time[0]
            vram = get_vram_gb()
            update_stage_info(current_stage[0], elapsed, vram)
        
        # Update current stage tracking
        if stage_num != current_stage[0]:
            current_stage[0] = stage_num
            stage_start_time[0] = time.time()
        
        # Call original progress callback
        progress_callback(stage_num, stage_name, progress_pct)
    
    results = run_pipeline(config, progress_callback=enhanced_progress_callback)
    
    # Update final stage info
    elapsed = time.time() - stage_start_time[0]
    vram = get_vram_gb()
    update_stage_info(current_stage[0], elapsed, vram)
    
    # Update header to complete
    update_header('v Pipeline Complete!', 'var(--color-success)')
    
    log_setup("Pipeline execution completed successfully", 'success', verbose)
    display_results(results)
    PIPELINE_COMPLETE = True
    
except Exception as e:
    # Update header to error
    update_header('x Pipeline Error', 'var(--color-error)')
    
    # Show error message
    display(HTML(f'<div style="color: var(--color-error); padding: 10px; background: rgba(255,0,0,0.1); border-radius: 4px; margin-top: 10px;"><b>Error:</b> {str(e)}</div>'))
    log_setup(f"Pipeline execution failed: {str(e)}", 'error', verbose)
    raise

In [ ]:
# @title ### View Results
# @markdown Display pipeline results in a rich, interactive format.

import sys
from pathlib import Path
from IPython.display import HTML, display

# ============================================================================
# Run basic checks
# ============================================================================
if 'SETUP_COMPLETE' not in globals() or not SETUP_COMPLETE:
    raise RuntimeError("Setup not completed. Please run the first cell to complete environment setup before viewing results.")

from library import load_template, log_setup, get_verbosity, show_pipeline_warning, is_colab_environment

if 'PIPELINE_COMPLETE' not in globals() or not PIPELINE_COMPLETE:
    show_pipeline_warning()

if 'results' not in globals():
    raise RuntimeError("Pipeline results not found. Please run the Run Pipeline cell first.")

# ============================================================================
# Display Results
# ============================================================================
verbose = get_verbosity()
log_setup("Displaying pipeline results", 'info', verbose)

metrics = results["performance_metrics"]
text_outputs = results["text_outputs"]
file_paths = results["file_paths"]

# Build stage metrics HTML
stages = [
    ("Audio Preprocessing", metrics.get("audio_preprocessing", {})),
    ("ASR Transcription", metrics.get("asr", {})),
    ("Translation", metrics.get("translation", {})),
    ("Summarization", metrics.get("summary", {})),
    ("Podcast Script", metrics.get("podcast_script", {})),
    ("TTS Synthesis", metrics.get("tts", {}))
]

stage_metrics_html = ""
for stage_name, stage_metrics in stages:
    if stage_metrics:
        time_val = stage_metrics.get("time_seconds", 0)
        vram_val = stage_metrics.get("peak_vram_gb", 0)
        stage_metrics_html += f'''
            <div class="stage-card">
                <h5>{stage_name}</h5>
                <p>Time: {time_val:.2f}s</p>
                <p>VRAM: {vram_val:.2f} GB</p>
            </div>
        '''

# Format numeric values for display
total_runtime_formatted = f"{metrics['total_runtime_seconds']:.2f}s"
audio_duration_formatted = f"{metrics.get('tts', {}).get('audio_duration_seconds', 0):.1f}s"
real_time_factor_formatted = f"{metrics.get('tts', {}).get('real_time_factor', 0):.2f}x"

# Get output directory
output_dir = file_paths.get("output_directory")
if not output_dir:
    audio_path = file_paths.get("final_podcast_audio", "")
    if audio_path:
        output_dir = str(Path(audio_path).parent)
    else:
        output_dir = "N/A"

# Create audio player HTML (using HTML5 audio with embedded data)
audio_path = file_paths.get("final_podcast_audio", "")
audio_player_html = ""
if audio_path and Path(audio_path).exists():
    # Use IPython Audio to create embedded player
    from IPython.display import Audio
    import base64
    with open(audio_path, 'rb') as f:
        audio_data = f.read()
    audio_b64 = base64.b64encode(audio_data).decode()
    audio_player_html = f'''
        <div style="padding: 0 24px;">
            <audio controls style="width: 100%; max-width: 600px;">
                <source src="data:audio/wav;base64,{audio_b64}" type="audio/wav">
                Your browser does not support the audio element.
            </audio>
        </div>
    '''

# Create download buttons HTML
files_to_download = [
    ("Original Transcript", file_paths.get("original_transcript", "")),
    ("Translated Transcript", file_paths.get("translated_transcript", "")),
    ("Key Points", file_paths.get("key_points_summary", "")),
    ("Podcast Script", file_paths.get("podcast_script", "")),
    ("Podcast Audio", file_paths.get("final_podcast_audio", ""))
]

is_colab = is_colab_environment()
buttons_html = '<div style="padding: 0 24px 24px 24px; display: flex; flex-wrap: wrap; gap: 8px;">'

for file_name, file_path in files_to_download:
    if file_path and Path(file_path).exists():
        if is_colab:
            # Create button that triggers Colab file download
            buttons_html += f'''
                <button onclick="google.colab.kernel.invokeFunction('download_{file_name.replace(' ', '_')}', [], {{}})"
                        style="background: #1a73e8; color: white; border: none; padding: 8px 16px;
                               border-radius: 20px; cursor: pointer; font-size: 14px; font-weight: 500;
                               box-shadow: 0 1px 2px rgba(0,0,0,0.05);">
                    {file_name}
                </button>
            '''
        else:
            # Create link for local download
            buttons_html += f'''
                <a href="{file_path}" download
                   style="background: #1a73e8; color: white; border: none; padding: 8px 16px;
                          border-radius: 20px; cursor: pointer; font-size: 14px; font-weight: 500;
                          text-decoration: none; display: inline-block;
                          box-shadow: 0 1px 2px rgba(0,0,0,0.05);">
                    {file_name}
                </a>
            '''

buttons_html += '</div>'

# Register download functions for Colab
if is_colab:
    from google.colab import output as colab_output
    for file_name, file_path in files_to_download:
        if file_path and Path(file_path).exists():
            def make_download_fn(fpath):
                def download_fn():
                    from google.colab import files
                    files.download(fpath)
                return download_fn
            colab_output.register_callback(f'download_{file_name.replace(" ", "_")}', make_download_fn(file_path))

# Load template and replace placeholders
html = load_template(
    'results_display.html',
    total_runtime=total_runtime_formatted,
    audio_duration=audio_duration_formatted,
    real_time_factor=real_time_factor_formatted,
    stage_metrics_html=stage_metrics_html,
    original_transcript=text_outputs["original_transcript"],
    translated_transcript=text_outputs["translated_transcript"],
    key_points_summary=text_outputs["key_points_summary"],
    podcast_script=text_outputs["podcast_script"],
    audio_path="",
    output_directory=output_dir,
    file_links_html="",
    audio_player_html=""
)

# Replace placeholders with HTML content
html = html.replace('<!-- AUDIO_WIDGET_PLACEHOLDER -->', audio_player_html)
html = html.replace('<!-- BUTTONS_WIDGET_PLACEHOLDER -->', buttons_html)

# Display the complete HTML
display(HTML(html))